In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width: 98% !important }</style>'))

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from utilities import *

In [3]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-2], '-')
    return [tf.strings.to_number(parts[-2]), tf.strings.to_number(parts[-1])]

In [4]:
def decode_img(img):
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [5]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [6]:
train_ds = tf.data.Dataset.list_files('../Data/train_images/*/*.png')

In [7]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [8]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(32)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [9]:
train_ds = prepare_for_training(train_ds)

In [10]:
test_ds = tf.data.Dataset.list_files('../Data/test_images/*/*.png')

In [11]:
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [12]:
def prepare_for_testing(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(32)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [13]:
test_ds = prepare_for_testing(test_ds)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(17, 34, 1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 15, 32, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 3840)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                122912    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 123,058
Trainable params: 123,058
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

history = model.fit(train_ds, epochs=200, steps_per_epoch=1000, validation_data=test_ds, validation_steps=250)

Train for 1000 steps, validate for 250 steps
Epoch 1/200
1000/1000 [==============================] - 5s 5ms/step - loss: 12.9075 - mae: 2.3225 - mse: 12.9075 - val_loss: 3.3529 - val_mae: 1.3957 - val_mse: 3.3529
Epoch 2/200
1000/1000 [==============================] - 4s 4ms/step - loss: 2.1697 - mae: 1.1498 - mse: 2.1697 - val_loss: 3.6543 - val_mae: 1.4533 - val_mse: 3.6543
Epoch 3/200
1000/1000 [==============================] - 3s 3ms/step - loss: 2.0385 - mae: 1.1076 - mse: 2.0385 - val_loss: 4.1670 - val_mae: 1.5458 - val_mse: 4.1670
Epoch 4/200
1000/1000 [==============================] - 3s 3ms/step - loss: 1.9065 - mae: 1.0652 - mse: 1.9065 - val_loss: 3.6234 - val_mae: 1.4580 - val_mse: 3.6234
Epoch 5/200
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8703 - mae: 1.0544 - mse: 1.8703 - val_loss: 3.7943 - val_mae: 1.4999 - val_mse: 3.7943
Epoch 6/200
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8348 - mae: 1.0399 - mse: 1.8348 - val_

1000/1000 [==============================] - 3s 3ms/step - loss: 0.4920 - mae: 0.5042 - mse: 0.4920 - val_loss: 2.5482 - val_mae: 1.2508 - val_mse: 2.5482
Epoch 50/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4858 - mae: 0.5030 - mse: 0.4858 - val_loss: 2.5811 - val_mae: 1.2636 - val_mse: 2.5811
Epoch 51/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4845 - mae: 0.5028 - mse: 0.4845 - val_loss: 2.7372 - val_mae: 1.2965 - val_mse: 2.7372
Epoch 52/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4816 - mae: 0.4983 - mse: 0.4816 - val_loss: 2.4791 - val_mae: 1.2363 - val_mse: 2.4791
Epoch 53/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4783 - mae: 0.4961 - mse: 0.4783 - val_loss: 2.6495 - val_mae: 1.2713 - val_mse: 2.6495
Epoch 54/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4691 - mae: 0.4934 - mse: 0.4691 - val_loss: 2.5425 - val_mae: 1.2574 - val_mse: 2.5425
Epoch

Epoch 98/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3995 - mae: 0.4457 - mse: 0.3995 - val_loss: 2.6754 - val_mae: 1.2800 - val_mse: 2.6754
Epoch 99/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3960 - mae: 0.4411 - mse: 0.3960 - val_loss: 2.7079 - val_mae: 1.2891 - val_mse: 2.7079
Epoch 100/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3957 - mae: 0.4418 - mse: 0.3957 - val_loss: 2.7154 - val_mae: 1.2876 - val_mse: 2.7154
Epoch 101/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3981 - mae: 0.4422 - mse: 0.3981 - val_loss: 2.6751 - val_mae: 1.2779 - val_mse: 2.6751
Epoch 102/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3946 - mae: 0.4419 - mse: 0.3946 - val_loss: 2.5330 - val_mae: 1.2528 - val_mse: 2.5330
Epoch 103/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3943 - mae: 0.4404 - mse: 0.3943 - val_loss: 2.5013 - val_mae: 1.2418 - val_

1000/1000 [==============================] - 3s 3ms/step - loss: 0.3002 - mae: 0.3723 - mse: 0.3002 - val_loss: 2.6714 - val_mae: 1.2821 - val_mse: 2.6714
Epoch 147/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2920 - mae: 0.3682 - mse: 0.2920 - val_loss: 2.7453 - val_mae: 1.2988 - val_mse: 2.7452
Epoch 148/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2902 - mae: 0.3661 - mse: 0.2902 - val_loss: 2.7512 - val_mae: 1.3048 - val_mse: 2.7512
Epoch 149/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2950 - mae: 0.3682 - mse: 0.2950 - val_loss: 2.6217 - val_mae: 1.2735 - val_mse: 2.6217
Epoch 150/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2856 - mae: 0.3610 - mse: 0.2856 - val_loss: 2.6789 - val_mae: 1.2862 - val_mse: 2.6789
Epoch 151/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2909 - mae: 0.3652 - mse: 0.2909 - val_loss: 2.8536 - val_mae: 1.3401 - val_mse: 2.8536


1000/1000 [==============================] - 3s 3ms/step - loss: 0.2601 - mae: 0.3395 - mse: 0.2601 - val_loss: 2.9244 - val_mae: 1.3367 - val_mse: 2.9244
Epoch 195/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2581 - mae: 0.3381 - mse: 0.2581 - val_loss: 2.7903 - val_mae: 1.3018 - val_mse: 2.7903
Epoch 196/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2577 - mae: 0.3379 - mse: 0.2577 - val_loss: 2.8164 - val_mae: 1.3014 - val_mse: 2.8164
Epoch 197/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2547 - mae: 0.3368 - mse: 0.2547 - val_loss: 2.8799 - val_mae: 1.3271 - val_mse: 2.8799
Epoch 198/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2604 - mae: 0.3379 - mse: 0.2604 - val_loss: 2.9766 - val_mae: 1.3510 - val_mse: 2.9766
Epoch 199/200
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2585 - mae: 0.3378 - mse: 0.2585 - val_loss: 2.9178 - val_mae: 1.3295 - val_mse: 2.9178
